In [64]:
"""
@Djibril ===> With stopword && With stemmer
@Omeima ===> With stopword && No stemmer
@Doua ===> No stopword && With stemmer
@Moh ===> No stopword && No stemmer
"""
from os import *
import practice2
import re
import practice3
import matplotlib.pyplot as plt
import pandas as pd
import math
import time
import random
from stop_words import get_stop_words



## sigles:
- wf = Weiting function (ltn,ltc,bm25)
- pl = Posting list{'term':[('docno','frequence')....]} 
- lt = Liste terme('Docno': '[list of terme]')
- stat = statistiques
- sw = Stopwords
- st = stemmer
- q = query

In [71]:
def build_directory():
    if not path.exists("assets"):
        mkdir('assets')
    if not path.exists("assets/runs_result/"):
        mkdir("assets/runs_result/")
# text_mining(fileName,use_stem=bool(),use_stopword=bool())
def create_pl_lt_stat_files(corpus_filename, use_stem,use_stopword):
    posting_list, list_terms,indexing_time = practice2.text_mining(filename,
                                                                    use_stem,
                                                                   use_stopword)
    if use_stopword:
        sw=''
    else:
        sw = 'no'
    if use_stem:
        st=''
    else:
        st = 'no'
    
    # Create the txt file to save the posting_list and liste_terme values
    create_write_file(posting_list,sw,st,True)
    create_write_file(list_terms,sw,st,False)
    
    #Get the statistique
    stat = practice2.get_statistics(posting_list, list_terms)
    return posting_list,list_terms,stat,indexing_time
    
def create_write_file(var_l,sw,st,pl):
    filename = open(build_filename(sw,st,pl),"w")
    for val in var_l.items():
        filename.write(str(val)+ "\n")
    filename.close()
    
def build_filename(sw,st,pl):
    type_name ="posting_list_"
    if not pl:
        type_name = "list_termes_"
    filedirectory = "assets/"
    filename = str(filedirectory+type_name+'{}stopword_{}stem.txt'.format(sw,st))
    print(str(filename)+' Created !!')
    return filename

# Get posting liste and list term from file
def get_pl_lt_from_file(use_stem,use_stopword):
    start = time.time()
    
    if use_stopword:
        sw=''
    else:
        sw = 'no'
    if use_stem:
        st=''
    else:
        st = 'no'
    pl_filename = build_filename(sw,st,True)
    lt_filename = build_filename(sw,st,False)
    
    posting_list = get_pl_dict_from_file(str(pl_filename),True)
    list_terms = get_pl_dict_from_file(str(lt_filename),False)
    stat = practice2.get_statistics(posting_list, list_terms)
    
    
    return posting_list,list_terms,stat,(time.time()-start)

# Build a dict(pl or lt) from scratch with a file
def get_pl_dict_from_file(filename,pl = bool()):
    filecontent = practice2.preprocesFile(str(filename))
    dict_file = {}
    content = []
    i=0
    for line in filecontent.split('\n'):
        for k in line.split(', ',1):
            i+=1
            if i%2==0:
                
                if pl:
                    content = rebuild_df(k)
                else:
                    content = re.sub(r'[^\w\s]', '', str(k)).split()
                # rsplit(')',1 )Remplace le dernier ) par rien
                dict_file[word] = content
            word = k.replace("(",'').replace('\'','')
    return dict_file

def rebuild_df(content):
    doc = re.sub(r'[^\w\s]', '', str(content))
    j = 0
    df = []
    for v in doc.split():
        j+=1
        if j%2==0:
            df.append((docn,int(v)))
        docn = v
    return df

# build run ict for the weiting function

def score(query_list,rsv_wf):
    score = []
    for query in query_list:
        query  = query.split(' ',1)
        score.append(result_query(query[0],practice3.rsv_score(query[1],rsv_wf),team_name))
    return score,(len(score) * len(score[0]))

# For all weigting function
def all_score_wf(query_list, ltn,ltc,bm25):
    start = time.time()
    score_ltn,l_ltn = score(query,ltn)
    print("Ltn time : ",time.time() -start)
    start = time.time()
    score_ltc,l_ltc = score(query,ltc)
    print("Ltc time : ",time.time() -start)
    start = time.time()
    score_bm25,l_bm25 = score(query,bm25)
    print("Bm25 time : ",time.time() -start)
    len_all_score = sum([l_ltn,l_ltc,l_bm25])
    
    return score_ltn,score_ltc,score_bm25,len_all_score

# 2009011 Q0 15343730 1 0.011317152113888167 Mathias /article[1]
def result_query(num_query, rsv_result,team_name):
    score = reverse_score(rsv_result)
    x = score
    if len(x)>1500:
        x = x[:1500]
    rsv_r = [(num_query,
              'Q0', x[i][0],i+1,
              round(x[i][1],5), team_name, 
              '/article[1]') for i in range(len(x))]
    return rsv_r

def reverse_score(score_result):
    score = sorted(score_result, reverse=True)
    score = [(doc, result) for result, doc in score]
    return score

# wf_index is [0,1,2]==['ltn','ltc','bm25']
def create_run_file(run_id,wf_index,use_stem,use_stopword,k,b):
    run_directory = "assets/runs_result/"
    run_id+=1
    team_name = "DjibrilMohamedOmaimaDouae"
    stem = ['nostem', 'porter', 'lovins', 'paice']
    wf = ['ltn', 'ltc', 'bm25']
    st = stem[0]
    sw = 'nostop'
    stopwords = get_stop_words('english')
    len_stop = ''
    if(use_stopword):
        sw = 'stop'
        len_stop = str(len(stopwords))
    if use_stem:
        st = stem[1]
                #name_Runid_wf_Granularity_use_Parameters.tx
    run_file_name = '{}_{}_{}_element_sec_p_{}{}_{}'.format(team_name,run_id,
                                                          wf[wf_index],sw,
                                                          len_stop,st)
    #Cas du bm25
    if (wf_index == 2):
        run_file_name = str(run_file_name+'_k{}_b{}'.format(k,b))
        
    run_file_path = str(run_directory+run_file_name+'.txt')
    return open(run_file_path,"w"),run_id

# index is [0,1,2]==['ltn','ltc','bm25']
def build_run_file(run_id,wf_score,index,use_stem,use_stopword,k,b):
    start = time.time()
    if index!=2:
        k=0.0
        b=0.0
    run_file_name,run_id = create_run_file(run_id,index,use_stem,use_stopword,k,b)
    for score in wf_score:
        for i in range(len(score)):
            run_score = str(score[i]).replace(',','').replace("'",'').replace('(','').replace(')','')
            run_file_name.write(run_score + "\n")  
    run_file_name.close()
    
    print("Execution time for the run {} is {}".format(run_id,time.time()-start))
    return run_id
# uniquement pour le bm25tuning for a random value:
def bm25_tuning_random(run_id,number_run,score_bm25,use_stem,use_stopword):
    for i in range(number_run):
        k = random.uniform(1, 2)
        b = random.uniform(0.1, 1)
        run_id = build_run_file(run_id, score_bm25,2,use_stem,use_stopword,k,b)
    print("Run number is {}".format(run_id))
    
def bm25_tuning(run_id,score_bm25,use_stem,use_stopword):
    k = 1.2
    b = 0.0
    for i in range(11):
        run_id = build_run_file(run_id, score_bm25,2,use_stem,use_stopword,k,b)
        b+=0.1
    k = 0.0
    b = 0.75
    for i in range(21):
        run_id = build_run_file(run_id, score_bm25,2,use_stem,use_stopword,k,b)
        k+=0.2
    print("Run number is {}".format(run_id))

In [72]:
# get the corpus file
directory = "Practice_03_data/"
list_data = practice2.list_file_data(directory)
filename = str(directory+list_data[0])


In [73]:
# get the query file
directory = "Practice_04_data/topics_M2DSC_7Q.txt"
#list_data_q = practice2.list_file_data(directory)
filename_q = directory #str(directory+list_data_q[0])

In [74]:
# Cell to initialise global variable
team_name = "DjibrilMohamedOmaimaDouae"
assets = "assets/"
k = 1.2
b = 0.75
use_stem = True
use_stopword = True
run_id = 0

In [75]:
query_list = practice2.preprocesFile(filename_q)
query = query_list.split("\n")

In [76]:
build_directory()

In [ ]:
# Exemple cell for only test

# Test 1 create a pl and lt without stopword and without stemmer
# pl,lt,stat,id_time = create_pl_lt_stat_files(filename, use_stem,use_stopword) # 4 bugs fixed

# Test 2 rebuild indexing listes from files
pl,lt,stat,id_time = get_pl_lt_from_file( use_stem,use_stopword) # 3 bugs fixed also indexing time too high:64.09


assets/posting_list_stopword_stem.txt Created !!
assets/list_termes_stopword_stem.txt Created !!


In [ ]:
# Cell to  print variable
id_time

In [11]:

#filecontent = practice2.preprocesFile(str(assets+"list_termes_stopword_stem.text"))
#content = filecontent.split('\n')[0].split(', ',1)[1]
#re.sub(r'[^\w\s]', '', str(content)).split()

Temps d'execution de recuperation de chaque donné
- T = True ----  F = false
- X1X2--> X1 = used stopword ---- X2 used stemmer
    * TT - 17.62372136116028
    * FT - 11.378056526184082
    * TF - 8.721906900405884
    * FF - 11.420051336288452

## Compute weiting function
*Don't forget to import practice3.py !!!*
_each wf has a index number to create easily the file name_ Dont forget the index number <== A revoir car pas tres pratique
* ltn = practice3.smart_ltn(pl,stat['n_doc'], stat['colec_freq'])
* ltc = practice3.smart_ltc(ltn) _if ltn is not define compute it before compute the ltc_
* bm25 = practice3.bm25(pl,stat,k,b)
* All function : ltn,ltc,bm25 = practice3.weinting_function(posting_list, stat,1.2,0.75)

In [30]:
# Test cell for wf funtions
# Test 1 compute all wf RSV
#ltn,ltc,bm25 = practice3.weinting_function(pl, stat,1.2,0.75) # no bugs
# Test 2 compute all wf score
# score_ltn,score_ltc,score_bm25,len_all_score = all_score_wf(query, ltn,ltc,bm25) #6 bugs fixed

# Test 3 build run file
# run_id = build_run_file(run_id, score_ltc,1,use_stem,use_stopword,k,b) # 1
# run_id = build_run_file(run_id, score_bm25,2,use_stem,use_stopword,k,b)
# For tuning
# bm25_tuning(run_id,40,score_bm25,use_stem,use_stopword)

In [ ]:
score_ltn,score_ltc,score_bm25,len_all_score = all_score_wf(query, ltn,ltc,bm25)

Next step, take all the case of posting list(with stopword, nostopword, stem,nostem) and share task to everyone to do one case done§!!!


In [ ]:
run_id = bm25_tuning(run_id,score_bm25,use_stem,use_stopword)